In [2]:
import pandas as pd
import math as m
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import numpy as np
from tqdm import tqdm_notebook
from sklearn import  metrics
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, log_loss, f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
import lightgbm as lgbm
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
%matplotlib inline

/home/vladimir/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.DataFrame.from_csv('final_data_to_AHAN.csv')

/home/vladimir/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [4]:

def get_data_model(train, n, target_col = 'target'):
    if n==1:
        print ('Already exists')
    else:
        train[target_col+'_'+str(n)] = np.nan
        train.loc[:,target_col+'_'+str(n)] = np.append(train[target_col][n-1:].values,[np.nan]*(n-1))

In [5]:

np.random.seed(42)
def predict(train, type_models, blend_type, test_days):
    for i in range(2, test_days+1):
        get_data_model(train, n=i)
    target_col = ["target"+'_'+str(x) for x in range(1,test_days+1)]
    target_col[0] = 'target'
    estimaters = []
    
    for i in range(test_days):
        k = i%(len(type_models))
        model = type_models[k]#RandomForestClassifier(max_depth=3)
        model.fit(pd.concat([train.drop(target_col, axis=1), train[target_col[i]]], axis=1).dropna().drop(target_col[i], axis=1),
                     train[target_col[i]].dropna())
        estimaters.append(model)
        
    pred_month = []
    for day in range(1, test_days+1):
        pred_day = []
        for i in range(day-1, test_days):
            pred_day.append(estimaters[i].predict_proba(train.iloc[[-(i+1)+day-1]].drop(target_col, axis=1))[0][0])
        pred_day = np.array(pred_day)
        pred_month.append(pred_day)
    pred_month = np.array(pred_month)
    
    preds = []
    if (blend_type == 'mean'):
        for i in range(test_days):
            preds.append(1-(pred_month[i].mean().round()))
    elif (blend_type == 'weights'):
        for i in range(test_days):
            if i == test_days-2:
                weights = np.array([0.7, 0.3])
            elif i == test_days-1:
                weights = np.array([1])
            else:
                weights = np.array([0.5, 0.2]+[(1-0.7)/(test_days-i-2)] *(test_days-i-2))
            preds.append(1-(pred_month[i]*weights).sum().round())
    return preds, pred_month

In [6]:

rand = 20
pred, month = predict(train[:], [XGBClassifier(seed=rand),
                                 RandomForestClassifier(max_depth=2, random_state=rand),
                                 KNN(),
                                 XGBClassifier(seed=rand),
                                 KNN()], 
                                blend_type = 'weights', test_days = 20)

/home/vladimir/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/vladimir/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
pred

[0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0]